### 문제 2-4 : 여행 계획 분석기

In [26]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint
import os
import json

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_T


In [27]:
# 출력 구조 정의
response_schemas = [
    ResponseSchema(name="destination", description="방문한 여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="예산"),
    ResponseSchema(name="rating", description="1-5점의 추천도"),
    ResponseSchema(name="activities", description="주요 활동들을 리스트로 출력")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

In [28]:
# 프롬프트 템플릿
template = """
다음 여행 후기 혹은 계획을 분석하세요. 후기(계획) 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [29]:
# 모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [30]:
# 테스트 리뷰 데이터
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요.
총 30만 원 정도 썼는데 해운대에서 바다 구경하고, 자갈치 시장에서 회 먹고, 감천문화마을도 구경했어요.
정말 만족스러운 여행이었습니다.
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

In [31]:
# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

In [32]:
json_output = json.dumps(output, indent=4, ensure_ascii=False)

print(json_output)

{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만 원",
    "rating": "4점",
    "activities": [
        "해운대 바다 구경",
        "자갈치 시장에서 회 먹기",
        "감천문화마을 구경"
    ]
}
